# Train a Simplicial Complex Autoencoder (SCA) with Coadjacency Message Passing Scheme (CMPS)


🟥 $\quad m_{y \rightarrow x}^{(r \rightarrow r'' \rightarrow r)} = M(h_{x}^{t, (r)}, h_{y}^{t, (r)},att(h_{x}^{t, (r)}, h_{y}^{t, (r)}),x,y,{\Theta^t}) \qquad \text{where } r'' < r < r'$

🟥 $\quad m_{y \rightarrow x}^{(r'' \rightarrow r)} = M(h_{x}^{t, (r)}, h_{y}^{t, (r'')},att(h_{x}^{t, (r)}, h_{y}^{t, (r'')}),x,y,{\Theta^t})$

🟧 $\quad m_x^{(r \rightarrow r)}  = AGG_{y \in \mathcal{L}\_\downarrow(x)} m_{y \rightarrow x}^{(r \rightarrow r)}$

🟧 $\quad m_x^{(r'' \rightarrow r)} = AGG_{y \in \mathcal{B}(x)} m_{y \rightarrow x}^{(r'' \rightarrow r)}$

🟩 $\quad m_x^{(r)}  = \text{AGG}\_{\mathcal{N}\_k \in \mathcal{N}}(m_x^{(k)})$

🟦 $\quad h_{x}^{t+1, (r)} = U(h_x^{t, (r)}, m_{x}^{(r)})$

Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

In [13]:
import torch
import numpy as np

import toponetx.datasets as datasets
from sklearn.model_selection import train_test_split

from topomodelx.nn.simplicial.sca_cmps import SCACMPS

If GPUs are available we will make use of them. Otherwise, we will use CPU.

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

## Import data ##

The first step is to import the dataset, shrec16, a benchmark dataset for 3D mesh classification. We then lift each graph into our domain of choice, a simplicial complex.

We also retrieve:
- input signals `x_0`, `x_1`, and `x_2` on the nodes (0-cells), edges (1-cells), and faces (2-cells) for each complex: these will be the model's inputs,
- a scalar classification label `y` associated to the simplicial complex.

In [15]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
scs = shrec["complexes"]

Loading shrec 16 small dataset...

done!


In [16]:
i_complex = 6
print(
    f"The {i_complex}th simplicial complex has {x_0s[i_complex].shape[0]} nodes with features of dimension {x_0s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_1s[i_complex].shape[0]} edges with features of dimension {x_1s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_2s[i_complex].shape[0]} faces with features of dimension {x_2s[i_complex].shape[1]}."
)

The 6th simplicial complex has 252 nodes with features of dimension 6.
The 6th simplicial complex has 750 edges with features of dimension 10.
The 6th simplicial complex has 500 faces with features of dimension 7.


## Preparing the inputs to test each message passing scheme:

#### Coadjacency Message Passing Scheme (CMPS):
This will require features from faces, and edges again, but outputs features on the edges. The first neighborhood matrix will be the level 2 lower Laplacian, $L_{\downarrow, 2}$, and the second neighborhood matrix will be the transpose of the incidence matrix of the faces, $B_{2}^T$.

In [17]:
laplacian_down_1_list = []
laplacian_down_2_list = []
incidence1_t_list = []
incidence2_t_list = []

for sc in scs:
    laplacian_down_1 = sc.down_laplacian_matrix(rank=1)
    laplacian_down_2 = sc.down_laplacian_matrix(rank=2)
    incidence_1_t = sc.incidence_matrix(rank=1).T
    incidence_2_t = sc.incidence_matrix(rank=2).T

    laplacian_down_1 = torch.from_numpy(laplacian_down_1.todense()).to_sparse()
    laplacian_down_2 = torch.from_numpy(laplacian_down_2.todense()).to_sparse()
    incidence_1_t = torch.from_numpy(incidence_1_t.todense()).to_sparse()
    incidence_2_t = torch.from_numpy(incidence_2_t.todense()).to_sparse()

    laplacian_down_1_list.append(laplacian_down_1)
    laplacian_down_2_list.append(laplacian_down_2)
    incidence1_t_list.append(incidence_1_t)
    incidence2_t_list.append(incidence_2_t)

# Create the Neural Networks

Using the SCACMPSLayer class, we create a neural network with a modifiable number of layers each following the CMPS at each level.

In [18]:
channels_list = [x_0s[0].shape[-1], x_1s[0].shape[-1], x_2s[0].shape[-1]]

complex_dim = 3

In [19]:
x_0s.shape

(100,)

# Train and Test Split

In [20]:
test_size = 0.2
x_0_train, x_0_test = train_test_split(x_0s, test_size=test_size, shuffle=False)
x_1_train, x_1_test = train_test_split(x_1s, test_size=test_size, shuffle=False)
x_2_train, x_2_test = train_test_split(x_2s, test_size=test_size, shuffle=False)

laplacian_down_1_train, laplacian_down_1_test = train_test_split(
    laplacian_down_1_list, test_size=test_size, shuffle=False
)
laplacian_down_2_train, laplacian_down_2_test = train_test_split(
    laplacian_down_2_list, test_size=test_size, shuffle=False
)
incidence1_t_train, incidence1_t_test = train_test_split(
    incidence1_t_list, test_size=test_size, shuffle=False
)
incidence2_t_train, incidence2_t_test = train_test_split(
    incidence2_t_list, test_size=test_size, shuffle=False
)

y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)
y_train.shape

(80,)

# Training and Testing Model
Because the SCA implementation in [HZPMC22]_ was used for clustering, we did the same in one dimension to train on the int classification labels provided my the shrec16 dataset. 

In [21]:
model = SCACMPS(
    channels_list=channels_list,
    complex_dim=complex_dim,
    n_classes=1,
    n_layers=3,
    att=False,
)
model = model.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.MSELoss()